In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import scipy.stats as stats
import statsmodels.api as sm
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import ConfusionMatrixDisplay
import scipy.stats as stats

# Set the matplotlib inline for displaying plots in Jupyter Notebook
%matplotlib inline


In [132]:
df = pd.read_csv("/kaggle/input/exam-still/claims_data.csv")

In [133]:
rand_dollar = pd.read_csv("/kaggle/input/exam-still/rand-dollar.csv")

In [134]:
data = pd.read_excel("/kaggle/input/exam-still/matches.xlsx")

In [135]:
football_players = pd.read_csv("/kaggle/input/exam-still/football_players.csv", encoding='latin1')

In [136]:
# Split data into X and y
X = rand_dollar.drop(columns=['ZAR/USD'])
y = rand_dollar['ZAR/USD']

# Print the shapes of X and y to confirm the separation
print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (120, 14)
y shape: (120,)


In [137]:
# Create a StandardScaler object
scaler = StandardScaler()

In [138]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [139]:
# Fit the scaler to the training data and transform both the training and testing data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

ValueError: could not convert string to float: '2008M01'

In [ ]:
# Train the linear regression model
# Create and train the model
lm = LinearRegression()
lm.fit(X_train[['Value of Exports (ZAR)']], y_train)

## Question 1

In [ ]:
# Print the intercept of the model
print("Intercept:", lm.intercept_)

## Question 3

In [ ]:
slope = lm.coef_[0]
print("Slope:", slope)

## Question 5

In [ ]:
# Define the value of exports
exports = 100000

# Reshape the exports value for prediction
exports_reshaped = np.array(exports).reshape(-1, 1)

# Predict the exchange rate
predicted_exchange_rate = lm.predict(exports_reshaped)
print("Predicted Exchange Rate:", predicted_exchange_rate[0])


## Question 6

In [ ]:
# Predict the target variable for the test set
y_pred = lm.predict(X_test[['Value of Exports (ZAR)']])

# Calculate the Mean Squared Error
mse = np.mean((y_pred - y_test) ** 2)
print("Mean Squared Error:", mse)


## Question 7

In [ ]:
# Calculate the residuals
residuals = y_test - y_pred

# Calculate the total sum of squares
tss = np.sum((y_test - np.mean(y_test)) ** 2)

# Calculate the residual sum of squares
rss = np.sum(residuals ** 2)

# Calculate the R-squared value
r_squared = 1 - (rss / tss)
print("R-squared value:", r_squared)


## Question 9

In [ ]:
# Define the actual value for August 2017
actual_value_aug_2017 = 103199.17

# Predict the exchange rate for August 2017
predicted_exchange_rate_aug_2017 = lm.predict([[actual_value_aug_2017]])
print("Predicted Exchange Rate for August 2017:", predicted_exchange_rate_aug_2017[0])


## Question 10

In [ ]:
# Calculate the absolute error
absolute_error = abs(predicted_exchange_rate_aug_2017 - 13.23)
print("Absolute Error for August 2017 prediction:", absolute_error)

## Question 11

In [ ]:
correlations = df.corr()['ZAR/USD'].abs().sort_values()
weakest_variable = correlations.index[0]
print("Variable with the weakest linear relationship with ZAR/USD:", weakest_variable)


## Question 12

In [ ]:
correlations = df.corr()['ZAR/USD'].abs().sort_values(ascending=False)
strongest_variable = correlations.index[1]
print("Variable with the strongest linear relationship with ZAR/USD:", strongest_variable)


## Question 13

In [ ]:
df_lasso_ridge = pd.read_csv('rand-dollar.csv', index_col=0)

X = df_lasso_ridge.drop(columns=['ZAR/USD'])
y = df_lasso_ridge['ZAR/USD']

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)


In [ ]:
# Train Ridge model
ridge = Ridge()
ridge.fit(X_train, y_train)


# Train Lasso model
lasso = Lasso(alpha=0.01)
lasso.fit(X_train, y_train)

# Predict the target variable for the training set using the Ridge model
ridge_y_pred_train = ridge.predict(X_train)

# Calculate the Mean Squared Error for the training set using the Ridge model
ridge_mse_train = mean_squared_error(y_train, ridge_y_pred_train)
print("Training MSE of Ridge model:", ridge_mse_train)

## Question 14

In [ ]:
# Calculate training MSE for Lasso model
lasso_y_pred = lasso.predict(X_train)
lasso_mse = mean_squared_error(y_train, lasso_y_pred)
print("Training MSE of Lasso model:", lasso_mse)

## Question 15

In [ ]:
# Predict the target variable for the test set using the Ridge model
ridge_y_pred_test = ridge.predict(X_test)

# Calculate the Mean Squared Error for the test set using the Ridge model
ridge_mse_test = mean_squared_error(y_test, ridge_y_pred_test)
print("Testing MSE of Ridge model:", ridge_mse_test)

## Question 16

In [ ]:
# Predict the target variable for the test set using the Lasso model
lasso_y_pred_test = lasso.predict(X_test)

# Calculate the Mean Squared Error for the test set using the Lasso model
lasso_mse_test = mean_squared_error(y_test, lasso_y_pred_test)
print("Testing MSE of Lasso model:", lasso_mse_test)

## Question 17

In [ ]:
# Get the absolute values of the coefficients of the Ridge model
ridge_coefficients = abs(ridge.coef_)

# Find the index of the indicator with the highest coefficient
best_predictor_index = ridge_coefficients.argmax()

# Get the name of the best predictor
best_predictor = X.columns[best_predictor_index]

print("Best predictor based on Ridge model's coefficients:", best_predictor)

## Question 18

In [ ]:
# Get the absolute values of the coefficients of the Ridge model
ridge_coefficients = abs(ridge.coef_)

# Find the index of the indicator with the lowest coefficient
worst_predictor_index = ridge_coefficients.argmin()

# Get the name of the worst predictor
worst_predictor = X.columns[worst_predictor_index]

print("Worst predictor based on Ridge model's coefficients:", worst_predictor)

## Question 19

In [ ]:
# Get the absolute values of the coefficients of the Lasso model
lasso_coefficients = abs(lasso.coef_)

# Find the index of the indicator with the highest coefficient
best_predictor_index = lasso_coefficients.argmax()

# Get the name of the best predictor
best_predictor = X.columns[best_predictor_index]

print("Best predictor based on Lasso model's coefficients:", best_predictor)

## Question 20

In [ ]:
# Count the number of coefficients equal to zero in the Lasso model
zero_coefficients_count = sum(lasso.coef_ == 0)

print("Number of variables with coefficients equal to zero in the Lasso model:", zero_coefficients_count)

## Question 21

In [ ]:
#Filter individuals with BMI greater than 25
overweight_or_obese = df[df['bmi'] > 25]

# Calculate proportion of individuals with BMI greater than 25
proportion_overweight_or_obese = len(overweight_or_obese) / len(df)

print("Proportion of individuals classified as overweight or obese (BMI > 25):", proportion_overweight_or_obese)

## Question 22

In [ ]:
# Calculate mean and variance of the 'children' column
mean_children = df['children'].mean()
var_children = df['children'].var()

print("Mean of 'children' column:", mean_children)
print("Variance of 'children' column:", var_children)

## Question 23

In [ ]:
# Calculate mean and standard deviation of the 'age' column from the dataset
mean_age = df['age'].mean()
std_age = df['age'].std()

# Calculate the probability of being aged 60 or older using the cumulative distribution function (CDF)
prob_60_or_older = 1 - stats.norm.cdf(60, loc=mean_age, scale=std_age)

# Number of individuals in the dataset
total_individuals = len(df['age'])

# Expected number of individuals aged 60 or older
expected_60_or_older = round(total_individuals * prob_60_or_older)

# Count of individuals aged 60 or older in the dataset
actual_60_or_older = sum(1 for age in df['age'] if age >= 60)

print("Expected number of individuals aged 60 or older:", expected_60_or_older)
print("Actual number of individuals aged 60 or older:", actual_60_or_older)


## Question 24

In [ ]:
# Create joint plot
joint = sns.jointplot(x='age', y='bmi', data=df, kind='scatter')

# Calculate correlation coefficient
correlation_coefficient = df['age'].corr(df['bmi'])

# Annotate the plot with correlation coefficient
joint.ax_joint.annotate(f'Corr: {correlation_coefficient:.2f}', xy=(40, 50), fontsize=12)

# Show the plot
plt.show()

## Question 25

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Assuming df contains the dataset with features and target variable

# Convert target variable to binary (1 indicates a claim)
df['insurance_claim'] = df['insurance_claim'].apply(lambda x: 1 if x == 'yes' else 0)

# Separate features and target variable
X = df.drop(['insurance_claim', 'claim_amount'], axis=1)
y = df['insurance_claim']

# Define categorical features
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

# Define preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ],
    remainder='passthrough'
)

# Define the logistic regression model
model = LogisticRegression()

# Create a pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(X_test)

# Calculate the proportion of correctly predicted claim indicators
accuracy = accuracy_score(y_test, y_pred)
print("Proportion of correctly predicted claim indicators:", accuracy)


## Question 26

In [ ]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Convert target variable to binary (1 indicates a claim)
df['insurance_claim'] = df['insurance_claim'].apply(lambda x: 1 if x == 'yes' else 0)

# Separate features and target variable
X = df.drop(['insurance_claim', 'claim_amount'], axis=1)
y = df['insurance_claim']

# Create dummy variables for categorical features
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
ct = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(drop='first'), categorical_features)],
    remainder='passthrough'
)
X = ct.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Add a constant to X matrices
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

# Fit logistic regression using statsmodels
logit_model = sm.Logit(y_train, X_train)
result = logit_model.fit()

# Print summary of the logistic regression model
print(result.summary())

## Question 28

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

# Convert target variable to binary (1 indicates a claim)
df['insurance_claim'] = df['insurance_claim'].apply(lambda x: 1 if x == 'yes' else 0)

# Separate features and target variable
X = df.drop(['insurance_claim', 'claim_amount'], axis=1)
y = df['insurance_claim']

# Create dummy variables for categorical features if needed
# Create dummy variables for categorical features
X = pd.get_dummies(X, drop_first=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Fit Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=101)
rf_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_classifier.predict(X_test)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)



# Extract TP, TN, FP, FN values
TN, FP, FN, TP = conf_matrix.ravel()

print("False Positives (FP):", FP)
print("False Negatives (FN):", FN)

## Question 29

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


# Convert target variable to binary (1 indicates a claim)
df['insurance_claim'] = df['insurance_claim'].apply(lambda x: 1 if x == 'yes' else 0)

# Separate features and target variable
X = df.drop(['insurance_claim', 'claim_amount'], axis=1)
y = df['insurance_claim']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Fit SVM models with different kernels
svm_radial = SVC(kernel='rbf')  # Radial kernel
svm_sigmoid = SVC(kernel='sigmoid')  # Sigmoid kernel
svm_linear = SVC(kernel='linear')  # Linear kernel

# Fit SVM models to the training data
svm_radial.fit(X_train, y_train)
svm_sigmoid.fit(X_train, y_train)
svm_linear.fit(X_train, y_train)

# Predictions
y_pred_radial = svm_radial.predict(X_test)
y_pred_sigmoid = svm_sigmoid.predict(X_test)
y_pred_linear = svm_linear.predict(X_test)

# Calculate accuracy scores
accuracy_radial = accuracy_score(y_test, y_pred_radial)
accuracy_sigmoid = accuracy_score(y_test, y_pred_sigmoid)
accuracy_linear = accuracy_score(y_test, y_pred_linear)

print("Accuracy score for radial kernel:", accuracy_radial)
print("Accuracy score for sigmoid kernel:", accuracy_sigmoid)
print("Accuracy score for linear kernel:", accuracy_linear)

## Question 30

In [ ]:
import pandas as pd

# Specify the file path
file_path = 'matches.xlsx'

# Read the Excel file into a DataFrame
data = pd.read_excel(file_path)

In [ ]:
data.head()

In [ ]:
# Calculate the proportion of matches with Duckworth-Lewis applied
dl_applied_proportion = data['dl_applied'].mean()

print(f"The proportion of matches with Duckworth-Lewis applied: {dl_applied_proportion:.2%}")

In [ ]:
#Filter matches where the team batting first is the winner
matches_won_by_batting_first = data[data['team1'] == data['winner']]

# Calculate the proportion of matches won by the team batting first
proportion_won_by_batting_first = len(matches_won_by_batting_first) / len(data)

print(f"The proportion of matches won by the team batting first: {proportion_won_by_batting_first:.2%}")

In [ ]:
# Convert the 'date' column to datetime format
data['date'] = pd.to_datetime(data['date'])
# 1. Create the feature indicating whether the match was played in April or not
data['is_april'] = data['date'].dt.month == 4



# 3. Count the number of April games and choices to field first
april_games_count = data['is_april'].sum()


print("Number of April games:", april_games_count)

# 1. Create the feature indicating whether the toss decision was to field or not
data['toss_decision_field'] = data['toss_decision'] == 'field'

# Count the number of times the toss decision was to field
field_decision_count = data['toss_decision_field'].sum()

print("Number of times the toss decision was to field:", field_decision_count)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score



# Convert the 'date' column to datetime format
data['date'] = pd.to_datetime(data['date'])

# 1. Create the feature indicating whether the match was played in April or not
data['is_april'] = data['date'].dt.month == 4

# 1. Create the feature indicating whether the toss decision was to field or not
data['toss_decision_field'] = data['toss_decision'] == 'field'

# Select the features and target variable
features = ['is_april', 'toss_decision_field']
target_variable = 'toss_winner'  # Replace with the actual target variable name
X = data[features]
y = data[target_variable]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=999)

# Build the Decision Tree classifier
decision_tree_classifier = DecisionTreeClassifier(random_state=999)
decision_tree_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = decision_tree_classifier.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the Decision Tree classifier:", accuracy)

## Question 35

In [ ]:
encodings = ['utf-8', 'latin1', 'ISO-8859-1']

for encoding in encodings:
    try:
        Fifa = pd.read_csv('football_players.csv', encoding=encoding)
        print("File read successfully with encoding:", encoding)
        break  # Stop trying encodings once successful
    except UnicodeDecodeError:
        print("Failed to read file with encoding:", encoding)

# Further processing of the Fifa DataFrame as needed

In [ ]:
Fifa.head()

In [ ]:
most_common_overall = Fifa['Overall'].value_counts().idxmax()

print("The most common Overall score for players in the database is:", most_common_overall)

## Question 36

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

# 1. Subset the dataset for central defenders
central_defenders = Fifa[Fifa['Preferred Positions'].str.contains('CB')]

# 2. Create a new target variable based on Overall score
def classify_overall(overall_score):
    if overall_score >= 80:
        return 'World Class'
    elif 70 <= overall_score < 80:
        return 'Good'
    else:
        return 'Mediocre'

central_defenders['Overall_Class'] = central_defenders['Overall'].apply(classify_overall)

# 3. Prepare data for modeling
X = central_defenders.select_dtypes(include=[np.number]).drop(['Overall'], axis=1)
y = central_defenders['Overall_Class']

# 4. Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1971)

# 5. Build the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=500, random_state=1971)
rf_classifier.fit(X_train, y_train)

# 6. Get feature importances
feature_importances = pd.Series(rf_classifier.feature_importances_, index=X.columns)

# 7. Display the top 5 important features
top_5_features = feature_importances.sort_values(ascending=False).head(5)
print("Top 5 important features:")
print(top_5_features)

## Question 38

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

# Assuming you have already prepared your dataset and defined your features (X) and target variable (y)

# 1. Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=911)

# 2. Define a list of k values to try
k_values = [1, 2, 3, 4, 5]

# 3. Initialize dictionaries to store F1 scores for each k value
f1_scores_world_class = {}
f1_scores_good = {}

# 4. Train KNN models with different k values and evaluate F1 scores
for k in k_values:
    # Train KNN model
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    
    # Predictions
    y_pred = knn.predict(X_test)
    
    # Calculate F1 scores for "World Class" and "Good" groups respectively
    f1_score_world_class = f1_score(y_test[y_test == 'World Class'], y_pred[y_test == 'World Class'], average='weighted')
    f1_score_good = f1_score(y_test[y_test == 'Good'], y_pred[y_test == 'Good'], average='weighted')
    
    # Store F1 scores in dictionaries
    f1_scores_world_class[k] = f1_score_world_class
    f1_scores_good[k] = f1_score_good

# 5. Determine the best k value for each group based on the highest F1 score
best_k_world_class = max(f1_scores_world_class, key=f1_scores_world_class.get)
best_k_good = max(f1_scores_good, key=f1_scores_good.get)

print("Best k value for World Class group:", best_k_world_class)
print("Best k value for Good group:", best_k_good)

## Question 41

In [ ]:
essay = pd.read_csv("/kaggle/input/essay-data/Essay_data.csv")

In [ ]:
essay.head(10)

In [ ]:
# Print the shape of the DataFrame before dropping missing values
print("Shape of DataFrame before dropping missing values:", essay.shape)

# Remove rows with missing values and reset index
essay = essay.dropna().reset_index(drop=True)

# Print the shape of the DataFrame after dropping missing values
print("Shape of DataFrame after dropping missing values:", essay.shape)


## Question 42

In [ ]:
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Original sentence
sentence = "I’m a part-time student @explore-software."

# Step 1: Remove stopwords
stop_words = set(stopwords.words('english'))
filtered_sentence = [word for word in sentence.split() if word.lower() not in stop_words]

# Step 2: Remove punctuation and replace by a single white space
punctuations = string.punctuation
cleaned_sentence = ''.join([char if char not in punctuations else ' ' for char in sentence])

# Step 3: Convert all text to lower case
lower_case_sentence = cleaned_sentence.lower()

# Tokenize the sentence into words using NLTK's word_tokenize function
words = word_tokenize(lower_case_sentence)

# Remove single-character words and "i'm"
words = [word for word in words if len(word) > 1 and word != "i’m"]

# Count bi-grams
bi_grams = list(ngrams(words, 2))

# Corrected count of bi-grams
correct_bi_gram_count = len(bi_grams)

# Print bi-grams and correct count
print("Bi-grams:", bi_grams)
print("Correct number of bi-grams:", correct_bi_gram_count)


## Question 43

In [ ]:
import pandas as pd
from fractions import Fraction

# Load the essay data
essay = pd.read_csv("/kaggle/input/essay-data/Essay_data.csv")

# Count the occurrences of 'N' and 'S' in the 'N/S' column
n_count = essay['N/S'].value_counts().get('N', 0)
s_count = essay['N/S'].value_counts().get('S', 0)

# Calculate the ratio of N students to S students
if s_count != 0:
    ratio_n_to_s = Fraction(n_count, s_count)
    print("The ratio of intuitive (N) students to sensing (S) students is:", f"{ratio_n_to_s.numerator}:{ratio_n_to_s.denominator}")
else:
    print("The ratio of intuitive (N) students to sensing (S) students is undefined because there are no sensing (S) students.")


## Question 44

In [ ]:
import pandas as pd
import re

# Load the essay data
essay_data = pd.read_csv("/kaggle/input/essay-data/Essay_data.csv")

# Function to remove punctuation and convert to lower case
def remove_punctuation_and_lower(text):
    # Remove punctuation using regular expression
    text_no_punctuation = re.sub(r'[^\w\s]', '', text)
    # Convert to lower case
    text_lower = text_no_punctuation.lower()
    return text_lower

# Apply the function to each essay in the 'Essay' column
essay_data['Essay'] = essay_data['Essay'].apply(remove_punctuation_and_lower)

# Display the updated DataFrame
print(essay_data.head())


In [ ]:
# Get the text of the first essay
first_essay_text = essay_data['Essay'][0]

# Get the 10th character (index 9) in the first essay
tenth_character = first_essay_text[9]

print("The 10th character in the first essay is:", tenth_character)


## Question 45

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

# Load the essay data
essay_data = pd.read_csv("/kaggle/input/essay-data/Essay_data.csv")

# Tokenize function
def tokenize_essay(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    return tokens

# Tokenize each essay in the 'Essay' column
essay_data['Tokenized_Essay'] = essay_data['Essay'].apply(tokenize_essay)

# Display the DataFrame with tokenized essays
print(essay_data[['Essay', 'Tokenized_Essay']].head())


In [ ]:
# Get the list of tokens for the 17th essay
tokens_17th_essay = essay_data['Tokenized_Essay'][16]

# Get the number of tokens in the list
num_tokens_17th_essay = len(tokens_17th_essay)

print("The number of tokens in the 17th essay is:", num_tokens_17th_essay)


## Question 47

In [ ]:
import nltk
from nltk.stem import SnowballStemmer

# Initialize SnowballStemmer for English
stemmer = SnowballStemmer('english')

# Stem the word "experiences"
stemmed_word = stemmer.stem('experiences')

print("Stemmed word:", stemmed_word)


## Question 48

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Load the essay data
essay_data = pd.read_csv("/kaggle/input/essay-data/Essay_data.csv")

# Tokenize function
def tokenize_essay(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    return tokens

# Remove stop words function
def remove_stop_words(tokens):
    # Get English stop words
    stop_words = set(stopwords.words('english'))
    # Remove stop words
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return filtered_tokens

# Tokenize and remove stop words for each essay
essay_data['Tokenized_Essay'] = essay_data['Essay'].apply(tokenize_essay)
essay_data['Filtered_Tokens'] = essay_data['Tokenized_Essay'].apply(remove_stop_words)

# Get the 24th token in the 81st essay
token_81st_essay = essay_data['Filtered_Tokens'][80][23]

print("The 24th token in the 81st essay is:", token_81st_essay)


## Question 49

In [ ]:
# Concatenate all filtered tokens from the essays
all_filtered_tokens = [token for tokens_list in essay_data['Filtered_Tokens'] for token in tokens_list]

# Get the number of unique words
num_unique_words = len(set(all_filtered_tokens))

print("The number of unique words in the essays after removing stopwords is:", num_unique_words)


## Question 50

In [ ]:
from collections import Counter

# Tokenize the text of the 56th essay
tokens_56th_essay = essay_data['Filtered_Tokens'][55]

# Create a bag of words (word counts)
bag_of_words_56th_essay = Counter(tokens_56th_essay)

# Get the count of 'time' in the 56th essay
count_time_56th_essay = bag_of_words_56th_essay.get('time', 0)  # Get the count of 'time', default to 0 if not found

print("The number of times 'time' was mentioned in the 56th essay is:", count_time_56th_essay)


## Question 51

In [ ]:
from collections import Counter

# Concatenate all filtered tokens from the essays
all_filtered_tokens = [token for tokens_list in essay_data['Filtered_Tokens'] for token in tokens_list]

# Count the occurrences of each word
word_counts = Counter(all_filtered_tokens)

# Count the number of unique words that appear at least twice
num_unique_words_at_least_twice = sum(1 for count in word_counts.values() if count >= 2)

# Count the total number of words in the essays
total_words = len(all_filtered_tokens)

# Calculate the percentage of unique words that appear at least twice
percentage_words_at_least_twice = (num_unique_words_at_least_twice / len(word_counts)) * 100

print("Percentage of words that appear at least twice in the essays:", percentage_words_at_least_twice)


## Question 52

In [ ]:
import string
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Filter essays by ENFJ personalities (J/P = J)
enfj_essays = essay_data.loc[essay_data['J/P'] == 'J', 'Essay']

# Concatenate all essays
enfj_text = ' '.join(enfj_essays)

# Tokenize the text and remove stopwords and punctuation
enfj_tokens = [word.lower() for word in word_tokenize(enfj_text) if word.lower() not in stopwords.words('english') and word.isalnum()]

# Count the occurrences of each word
word_counts = Counter(enfj_tokens)

# Find the most commonly mentioned word
most_common_word = word_counts.most_common(1)[0][0]

print("The most commonly mentioned word by ENFJ personalities after removing stop words and punctuation is:", most_common_word)


## Question 53

In [ ]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Define a function to generate bi-grams from text after removing stopwords and punctuation
def generate_bigrams(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())  # Convert text to lowercase
    
    # Remove stop words and punctuation
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
    
    # Generate bi-grams
    bigrams = [(filtered_tokens[i], filtered_tokens[i+1]) for i in range(len(filtered_tokens)-1)]
    return bigrams

# Apply the function to the 70th essay and store the bi-grams
bi_grams_70th_essay = generate_bigrams(essay_data.loc[70, 'Essay'])

# Get the 109th bi-gram in the 70th essay
bi_gram_109_70_essay = bi_grams_70th_essay[109]

print("The 109th bi-gram in the 70th essay after removing stopwords and punctuation is:", bi_gram_109_70_essay)
